# Doing inference with SageMaker Built-in Object Detection model

1. [Download the trained model artifact](#download)
1. [Convert training model to deployable model](#convert)
1. [Inference](#inference)
  1. [model load](#load)
  1. [single image inference](#singleinference)
  1. [batch inference](#batchinference)


## setup

In [ ]:
%matplotlib inline

In [ ]:
!pip install opencv-python
!pip install gluoncv

In [ ]:
from matplotlib import pyplot as plt
from gluoncv.utils import download, viz
import numpy as np
import mxnet as mx
import json
import boto3
import cv2

# Download the trained model artifact <a id='download'></a>

The trained model parameters along with its network definition is stored in a tar.gz file in the output path for the training job. We need to download and unzip it to local disk:

In [ ]:
JOB_ID='od-demo-2020-01-24-05-26-12' 
S3_OUTPUT_BUCKET = 'robcost-potatohead'

In [ ]:
sagemaker_client =  boto3.client('sagemaker')
MODEL_ARTIFACT = sagemaker_client.describe_training_job(TrainingJobName=JOB_ID)['ModelArtifacts']['S3ModelArtifacts']
MODEL_ARTIFACT

In [ ]:
import os 

def make_tmp_folder(folder_name):
    try:
        os.makedirs(folder_name)
    except OSError as e:
        print("{} folder already exists".format(folder_name))

In [ ]:
TMP_FOLDER = 'trained-model'
make_tmp_folder(TMP_FOLDER)

!aws s3 cp $MODEL_ARTIFACT $TMP_FOLDER/.

Unzipping the model you will find three files in your directory:
```
model_algo_1-symbol.json   <-- neural network definition 
hyperparams.json           <-- hyper parameters  
model_algo_1-0000.params   <-- trained weights for the neural network
```

In [ ]:
!tar -xvzf $TMP_FOLDER/model.tar.gz -C $TMP_FOLDER/

# Convert the training model to a deployable model <a id='convert'></a>

The model output produced by the built-in object detection model leaves the loss layer in place and does not include a [non-max suppression (NMS) layer](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH). To make it ready for inference on our machine, we need to remove the loss layer and add the NMS layer. We will be using a script from this GitHub repo: https://github.com/zhreshold/mxnet-ssd

Make sure to clone this Git repo to your ~/SageMaker folder

```
cd ~/SageMaker
git clone https://github.com/zhreshold/mxnet-ssd.git
```

In [ ]:
%%sh
cd ~/SageMaker
git clone https://github.com/zhreshold/mxnet-ssd.git

You need to run the `deploy.py` script to convert a trained model to a deployable model. I  found that you must use python2 to run this script successfully.

In [ ]:
!python /home/ec2-user/SageMaker/mxnet-ssd/deploy.py -h

When running this script, you need to make sure that command line options you pass in match exactly the hyperparameters of your training job. If you’re unsure, refer the hyperparams.json file in your unpacked model artifacts to confirm. 

In [ ]:
!cat $TMP_FOLDER/hyperparams.json

In [ ]:
!python /home/ec2-user/SageMaker/mxnet-ssd/deploy.py --network resnet50 --num-class 1 --nms .45 --data-shape 512 --prefix $TMP_FOLDER/model_algo_1

In [ ]:
!ls -alh $TMP_FOLDER

Save the copy of the deployable model artifact in S3

In [ ]:
!aws s3 cp $TMP_FOLDER/deploy_model_algo_1-0000.params s3://$S3_OUTPUT_BUCKET/deployable-model/
!aws s3 cp $TMP_FOLDER/deploy_model_algo_1-symbol.json s3://$S3_OUTPUT_BUCKET/deployable-model/

# 3. Doing inference with the model on local host <a id='inference'></a>

Below code will run inference on a set of test images on the current notebook instance. Using a GPU instance (e.g. p2.\*, p3.\* family) will result in faster performance than CPU only instances. You can stop the SageMaker notebook instance and update the instance type, and restart the notebook instance before continuing. 

In [ ]:
def get_ctx():
    try:
        gpus = mx.test_utils.list_gpus()
        if len(gpus) > 0:
            ctx = []
            for gpu in gpus:
                ctx.append(mx.gpu(gpu))
        else:
            ctx = [mx.cpu()]
    except:
        ctx = [mx.cpu()]
    return ctx

ctx = get_ctx()[0]
ctx

In [ ]:
SHAPE = 512
input_shapes=[('data', (1, 3, SHAPE, SHAPE))]
confidence_threshold = 0.3
CLASSES = ['potatohead']


## 3A. Loading the model <a id="load"> </a>

In [ ]:
%%time

param_path=os.path.join(TMP_FOLDER, 'deploy_model_algo_1')
print("param_path: {}".format(param_path))
sym, arg_params, aux_params = mx.model.load_checkpoint(param_path, 0)
mod = mx.mod.Module(symbol=sym, label_names=[], context=ctx)
mod.bind(for_training=False, data_shapes=input_shapes)
mod.set_params(arg_params, aux_params)

In [ ]:
from collections import namedtuple
Batch = namedtuple('Batch', ['data'])

def predict_from_file(filepath, reshape=(SHAPE, SHAPE)):
    # Switch RGB to BGR format (which ImageNet networks take)
    img = cv2.cvtColor(cv2.imread(filepath), cv2.COLOR_BGR2RGB)
    if img is None:
        return []

     # Resize image to fit network input
    img = cv2.resize(img, reshape)
    
    org_image = img.copy()
    img = np.swapaxes(img, 0, 2)
    img = np.swapaxes(img, 1, 2)
    img = img[np.newaxis, :]
 
    mod.forward(Batch([mx.nd.array(img)]))
    prob = mod.get_outputs()[0].asnumpy()
    prob = np.squeeze(prob)

    return prob, org_image

In [ ]:
def infer(image_path, threshold=confidence_threshold):
    results, org_image = predict_from_file(image_path)
    image_name = image_path.split("/")[-1]
    
    filtered_result = results[results[:, 0] != -1]
    filtered_result = filtered_result[filtered_result[:, 1] >=threshold]
    
    return filtered_result, org_image

## 3B. Test inference on single image <a id="singleinference"></a>

In [ ]:
!aws s3 cp s3://robcost-potatohead/dev/20191212_081328709_iOS.jpg .

In [ ]:
%%time

prediction_image = '20191212_081328709_iOS.jpg'
results, org_image = infer(prediction_image)

In [ ]:
results[:, (2, 4)] *= SHAPE
results[:, (3, 5)] *= SHAPE

In [ ]:
org_image.shape

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(15, 10), facecolor='white', dpi=100)
axis=axes

ax = viz.plot_bbox(org_image, results[:,-4:], results[:,1], results[:,0],thresh=0.1, class_names=CLASSES, ax=axis)

## 3C. Run inference on a batch of test images (~2 minutes on a p3.2xlarge instance, longer on cpu instances for 469 images) <a id="singleinference"></a>

In [ ]:
VALIDATION_IMAGE_FOLDER = 'validation'
make_tmp_folder(VALIDATION_IMAGE_FOLDER)

In [ ]:
!aws s3 sync s3://greengrass-object-detection-blog/frames/validation_box_video/ $VALIDATION_IMAGE_FOLDER/ --quiet

In [ ]:
!ls -1 $VALIDATION_IMAGE_FOLDER/ | wc -l

In [ ]:
%%time 

img_list = os.listdir(VALIDATION_IMAGE_FOLDER)

output_file = 'validation-inference-results.json'
with open(output_file, "w") as outfile:
    for img in img_list:
        if img.endswith("jpg"):
            result,_orig = infer(os.path.join(VALIDATION_IMAGE_FOLDER, img))
            to_write = {"image": img, "prediction": result.tolist()}
            outfile.write(json.dumps(to_write))
            outfile.write('\n')

In [ ]:
!head $output_file


In [ ]:
visualization_pdf_name = 'validation-visualization.pdf'

In [ ]:
%%time 

%run ./visualize_prediction_labels_batch.py -i $VALIDATION_IMAGE_FOLDER -l $output_file -f $visualization_pdf_name -c $confidence_threshold

In [ ]:
!aws s3 cp $visualization_pdf_name s3://$S3_OUTPUT_BUCKET/prediction-visualization/$JOB_ID/

You can open the [validation visualization pdf](./validation-visualization.pdf) on the notebook instance, or download the visualization PDF to your laptop to examine the prediction visualizaiton. e.g. 
```
aws s3 cp s3://greengrass-object-detection-blog/prediction-visualization/od-demo-2019-08-01-04-57-12/validation-visualization.pdf .
```